<h2>Dependency Installation</h2>

In [124]:
import tensorflow as tf
import pandas as pd
import numpy as np

<h3>Data Cleaning</h3>

In [125]:
applications = pd.read_csv('Application Data.csv')
applications.dropna(subset=['Username'], inplace=True)

In [126]:
applications_ = pd.DataFrame()

In [127]:
def ConvertFound(found: str) -> str:
    if found == 'Shouts':
        return 'Shout'
    elif found == 'Friend' or found == 'Other':
        return 'Other'
    elif found == 'Forums' or found == 'Wynn Discord':
        return 'External'
    elif found == 'In-game Party' or found == 'Map':
        return 'In-Game'
    else:
        return -1

In [128]:
def ConvertActivity(activity: str) -> str:
    if activity == 'No life' or activity == 'Daily':
        return activity
    elif activity == 'Sometimes' or activity == 'Weekly':
        return 'Other'
    else:
        return -1

In [129]:
def ConvertPreviousGuild(previousGuild: str) -> bool:
    if previousGuild == 'Rando' or previousGuild == 'Allies' or previousGuild == 'Attacker' or previousGuild == 'Adonis':
        return True
    elif np.isnan(previousGuild):
        return False
    else:
        return -1

In [130]:
def CountGuildGrindContributions() -> pd.Series:
    guildGrind = ['Warring', 'Grinding', 'Guild Objective']
    cont_1 = applications['Contribute 1'].apply(lambda x: 1 if x in guildGrind else 0)
    cont_2 = applications['Contribute 2'].apply(lambda x: 1 if x in guildGrind else 0)
    cont_3 = applications['Contribute 3'].apply(lambda x: 1 if x in guildGrind else 0)
    cont_4 = applications['Contribute 4'].apply(lambda x: 1 if x in guildGrind else 0)
    
    return cont_1 + cont_2 + cont_3 + cont_4

In [131]:
def CountCommunityContributions() -> pd.Series:
    community = ['Helping Others', 'Chatting']
    cont_1 = applications['Contribute 1'].apply(lambda x: 1 if x in community else 0)
    cont_2 = applications['Contribute 2'].apply(lambda x: 1 if x in community else 0)
    cont_3 = applications['Contribute 3'].apply(lambda x: 1 if x in community else 0)
    cont_4 = applications['Contribute 4'].apply(lambda x: 1 if x in community else 0)

    return cont_1 + cont_2 + cont_3 + cont_4

In [132]:
def CountActivities() -> pd.Series:
    activity_1 = applications['Enjoy 1'].isna().astype(int)
    activity_2 = applications['Enjoy 2'].isna().astype(int)
    activity_3 = applications['Enjoy 3'].isna().astype(int)

    return activity_1 + activity_2 + activity_3

In [133]:
def CountHobbies() -> pd.Series:
    hobby_1 = applications['Hobbies 1'].isna().astype(int)
    hobby_2 = applications['Hobbies 2'].isna().astype(int)
    hobby_3 = applications['Hobbies 3'].isna().astype(int)
    hobby_4 = applications['Hobbies 4'].isna().astype(int)

    return hobby_1 + hobby_2 + hobby_3 + hobby_4

In [134]:
applications_ = pd.DataFrame()

applications_['Age'] = applications['Age'].fillna(round(applications['Age'].mean()))
applications_['Activity'] = applications['Activity'].apply(ConvertActivity)
applications_['Found'] = applications['Found'].apply(ConvertFound)
applications_['Past Guild'] = applications['Prev guilds'].apply(ConvertPreviousGuild)

applications_['Guild Grind'] = CountGuildGrindContributions().apply(lambda x: True if x > 0 else False)
applications_['Community'] = CountCommunityContributions().apply(lambda x: True if x > 0 else False)

applications_['Num Enjoy'] = CountActivities()
applications_['Num Hobbies'] = CountHobbies()

applications_['Successful'] = applications['Successful'].map({'Yes': True, 'No': False, 'Trouble': False, 'Unknown':-1})

In [135]:
applications_.sample(n=20)

,Age,Activity,Found,Past Guild,Guild Grind,Community,Num Enjoy,Num Hobbies,Successful
473,18.0,Daily,Other,False,False,False,0,3,-1
263,18.0,Other,Shout,False,False,False,2,2,False
25,20.0,Daily,External,False,False,True,2,3,False
439,18.0,Daily,Shout,True,True,True,1,1,False
7,18.0,Other,Shout,False,False,False,2,3,False
138,18.0,Daily,Shout,False,False,False,2,2,False
145,19.0,Daily,Shout,False,False,True,2,2,False
198,18.0,Daily,Shout,False,False,False,2,1,False
178,19.0,Daily,Shout,True,True,False,0,0,False
203,20.0,Daily,Shout,False,False,True,2,1,True
